## Data Wrangling Step

This step involves the cleaning, and formatting of data for the purpose of
1. Data Analysis.
2. Machine Learning Modelling.
3. Development of actionable insights.



Data Should be downloaded and stored on the local machine or can be downloaded straight from the UCI repository depending on the preference.


In [ ]:
BASE_FOLDER = "/content/drive/MyDrive/Colab_Notebooks/online_retail/"

In [ ]:
# importing necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.graph_objects as go

import scipy as sp
import scipy.stats


In [ ]:
# Import Retail data
retail_data = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/online_retail/online_retail_II.csv")

In [ ]:
retail_data.tail()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France
1067370,581587,POST,POSTAGE,1,2011-12-09 12:50:00,18.00,12680.0,France


In [ ]:
# To explore the data and give a descriptive statistics of the data
retail_data.describe()

,Quantity,Price,Customer ID
count,1.067371e+06,1.067371e+06,824364.000000
mean,9.938898e+00,4.649388e+00,15324.638504
std,1.727058e+02,1.235531e+02,1697.464450
min,-8.099500e+04,-5.359436e+04,12346.000000
25%,1.000000e+00,1.250000e+00,13975.000000
50%,3.000000e+00,2.100000e+00,15255.000000
75%,1.000000e+01,4.150000e+00,16797.000000
max,8.099500e+04,3.897000e+04,18287.000000


From the above descriptive statistics it can be observed that certain values are abnormal

1. For instance the Quantity and Price both have negative values.

2. The CustomerID does not uniquely represent all the transactions that were carried out.

There are also other issues to be addressed.

3. Some descriptions are missing, this would also need to be sorted out as this will be useful for the analysis of the data.

# Data Cleaning tasks

In [ ]:
retail_data.isnull().sum()

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64

In [ ]:
print("Percentage of customers missing: ", round(retail_data['Customer ID'].isnull().sum() * 100 / len(retail_data),2),"%" )

Percentage of customers missing:  22.77 %


From the look of things its the description and the customer ID that possess the missing data. This rows with this missing data makes up 22.77% of the data. It therefore important to devise a means to fill in this missing data intelligently.

The average amount of orders made can be calculated using the invoice numbers and the total number of orders

In [ ]:
print("Number of orders in df1:", retail_data['Invoice'].nunique())
print("Avg. number of products per order in the dataset: ", round(retail_data['StockCode'].count() / retail_data['Invoice'].nunique(),2) )

Number of orders in df1: 53628
Avg. number of products per order in the dataset:  19.9


In the step to design or develop new customer IDs. This would ensure all customers can be uniquely identified.

In [ ]:
retail_data["Null_CustomerID"] = [1 if i==True else 0 for i in retail_data["Customer ID"].isnull()]

It is safe to assume that creating new customer IDs will not affect the insights we hope to gain from the data. Reason being that the invoice number can be used to identify all unique orders, which is our purpose of analysis for now.

New customer IDs can be assigned by using the invoice orders, however this would mean that customers for which their customer IDs were not captured are automatically seen as new and unique customers.

In [ ]:
retail_data["New_ID"] = retail_data["Customer ID"]
retail_data.loc[retail_data["Null_CustomerID"] == 1, ["New_ID"]] = retail_data["Invoice"]

In [ ]:
retail_data[retail_data["Null_CustomerID"] == 1].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Null_CustomerID,New_ID
263,489464,21733,85123a mixed,-96,2009-12-01 10:52:00,0.00,NaN,United Kingdom,1,489464
283,489463,71477,short,-240,2009-12-01 10:52:00,0.00,NaN,United Kingdom,1,489463
284,489467,85123A,21733 mixed,-192,2009-12-01 10:53:00,0.00,NaN,United Kingdom,1,489467
470,489521,21646,NaN,-50,2009-12-01 11:44:00,0.00,NaN,United Kingdom,1,489521
577,489525,85226C,BLUE PULL BACK RACING CAR,1,2009-12-01 11:49:00,0.55,NaN,United Kingdom,1,489525


In [ ]:
retail_data[retail_data["Null_CustomerID"] == 0].head() 

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Null_CustomerID,New_ID
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,0,13085
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,0,13085
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,0,13085
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,0,13085
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,0,13085


In [ ]:
retail_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 10 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   Invoice          1067371 non-null  object 
 1   StockCode        1067371 non-null  object 
 2   Description      1062989 non-null  object 
 3   Quantity         1067371 non-null  int64  
 4   InvoiceDate      1067371 non-null  object 
 5   Price            1067371 non-null  float64
 6   Customer ID      824364 non-null   float64
 7   Country          1067371 non-null  object 
 8   Null_CustomerID  1067371 non-null  int64  
 9   New_ID           1067371 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 81.4+ MB


In [ ]:
#drop Customer ID column since it isn't need anymore
retail_data = retail_data.drop('Customer ID', 1)
retail_data.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Country,Null_CustomerID,New_ID
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,United Kingdom,0,13085
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,United Kingdom,0,13085
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,United Kingdom,0,13085
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,United Kingdom,0,13085
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,United Kingdom,0,13085


To ensure consistency the data types of the new ID is meant to be numeric, since cancelled orders were inclusive in the invoice column some of the rows must have affected the data type of the New ID, 


In [ ]:
#Remove all non digits from column NewID
retail_data['New_ID'] = retail_data['New_ID'].astype(str).str.replace('\D+', '')

#Convert to integer
retail_data['New_ID'] = pd.to_numeric(retail_data['New_ID'])

#Check if the 'C' character was removed in the NewID column
retail_data[retail_data['New_ID']== 580605]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Country,Null_CustomerID,New_ID
1050063,C580605,AMAZONFEE,AMAZON FEE,-1,2011-12-05 11:36:00,17836.46,United Kingdom,1,580605


In [ ]:
retail_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 9 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   Invoice          1067371 non-null  object 
 1   StockCode        1067371 non-null  object 
 2   Description      1062989 non-null  object 
 3   Quantity         1067371 non-null  int64  
 4   InvoiceDate      1067371 non-null  object 
 5   Price            1067371 non-null  float64
 6   Country          1067371 non-null  object 
 7   Null_CustomerID  1067371 non-null  int64  
 8   New_ID           1067371 non-null  int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 73.3+ MB


In [ ]:

retail_data.isnull().sum()

Invoice                 0
StockCode               0
Description          4382
Quantity                0
InvoiceDate             0
Price                   0
Customer ID        243007
Country                 0
Null_CustomerID         0
New_ID                  0
dtype: int64

From the above result it is evident that all the transactions now possess unique customer IDs.

To aid the analysis of this data, other columns will also need be addressed. The columns would be addressed in this order:

1. Description column:
*   Ensure all items possess valid descriptions.
* Ensure all transactions posses descriptions.

2. Invoice Date
* Ensure Proper date-time formatting

3. StockCode
* Ensure proper StockCode assignments.
* Removing irrelevant StockCodes.

4. Country
* Ensure unique values are assigned in the Country column.

5. Price
* Remove outliers



In [ ]:
# Dealing with the Description columns
# String formatting

#Strings
retail_data['Description'] = retail_data['Description'].str.replace('.','').str.upper().str.strip()
retail_data['Description'] = retail_data['Description'].replace('\s+',' ',regex = True)

# Assigning correct and missing descriptions by using the StockCode
for index,value in retail_data.StockCode[retail_data.Description.isna()==True].items():
    if pd.notna(retail_data.Description[retail_data.StockCode == value]).sum() != 0:
        retail_data.Description[index] = retail_data.Description[retail_data.StockCode == value].mode()[0]
    else:
        retail_data.drop(index = index, inplace = True)
        
retail_data['Description'] = retail_data['Description'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
# Dealing with the StockCode column
# String formatting
retail_data['StockCode'] = retail_data['StockCode'].str.upper()

# Dropping all stockcodes that contain only strings
code_types = list(map(lambda codes: any(char.isdigit() for char in codes), retail_data['StockCode']))
irrelevant_codes = [i for i,v in enumerate(code_types) if v == False]
retail_data.drop(irrelevant_codes , inplace = True)



In [ ]:
retail_data.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country', 'Null_CustomerID', 'New_ID'],
      dtype='object')

In [ ]:
# Removing Outliers Based on Z-score
retail_data = retail_data[(np.abs(sp.stats.zscore(retail_data['Price']))<3) & (np.abs(sp.stats.zscore(retail_data['Quantity']))<5)]

# Dealing with missing & incorrect values in Price and Quantity columns
retail_data.drop(retail_data[(retail_data.Quantity>0) & (retail_data.Invoice.str.contains('C') == True)].index, inplace = True)
retail_data.drop(retail_data[(retail_data.Quantity<0) & (retail_data.Invoice.str.contains('C') == False)].index, inplace = True)
retail_data.drop(retail_data[retail_data.Description.str.contains('?',regex=False) == True].index, inplace = True)
retail_data.drop(retail_data[retail_data.Price == 0].index, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is 

In [ ]:
# Dealing with the country column

# Customers with Different Countries
Customers = retail_data.groupby('New_ID')['Country'].unique()
Customers.loc[Customers.apply(lambda x:len(x)>1)]

New_ID
123700           [Austria, Cyprus]
123940          [Belgium, Denmark]
124130             [Spain, France]
124170            [Belgium, Spain]
124220    [Australia, Switzerland]
124230          [Denmark, Belgium]
124290          [Denmark, Austria]
124310        [Belgium, Australia]
124490          [Denmark, Belgium]
124550             [Cyprus, Spain]
124570       [Switzerland, Cyprus]
126520           [Germany, France]
127450      [EIRE, United Kingdom]
Name: Country, dtype: object

In [ ]:
# Fixing duplicate CustomerIDs and Multiple Countries
for i,v in retail_data.groupby('New_ID')['Country'].unique().items():
    if len(v)>1:
        retail_data.Country[retail_data['New_ID'] == i] = retail_data.Country[retail_data['New_ID'] == i].mode()[0]

In [ ]:
# Adding Desired Features
retail_data['Amount_Spent'] = retail_data['Quantity']*retail_data['Price'] # Total 
# Formatting Date/Time
retail_data['InvoiceDate'] = pd.to_datetime(retail_data['InvoiceDate'], format = '%Y/%m/%d %H:%M')

retail_data['InvoiceMonth'] = retail_data['InvoiceDate'].apply(lambda x: x.strftime('%B'))

retail_data['Day of week'] = retail_data['InvoiceDate'].dt.day_name()

In [ ]:
# Exporting Processed Data
retail_data.to_csv(f'{BASE_FOLDER}cleaned_retail_data.csv', date_format = '%Y-%m-%d %H:%M', index = False)